<a href="https://colab.research.google.com/github/vu-topics-in-big-data-2023/Team08/blob/main/incident_join.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
credentials = {
    'region_name': 'us-east-1',
    'aws_access_key_id': 'xxx',
    'aws_secret_access_key': 'xxx',
    'aws_session_token': 'xxx'
}

In [ ]:
!pip install boto3
!pip install geopandas pyarrow

import geopandas as gd
import pyarrow.parquet as pq
import pandas as pd
import boto3, json

session = boto3.session.Session(**credentials)
s3 = session.client('s3')

In [3]:
session = boto3.session.Session(**credentials)
s3 = session.client('s3')

## Upload census tracts to S3

In [ ]:
import os

for root,dirs,files in os.walk('/content/census_tract'):
  for file in files:
    s3.upload_file(os.path.join(root[9:],file),'wwu-incidents', file)

## Join census tract, economics data, and incidents

In [ ]:
! unzip /content/census_tract.zip
df = gd.read_file('/content/census_tract/davidson/tl_2016_47_tract.shp')
df.head()

In [ ]:
df2 = pd.read_csv('/content/census_economic_data_all_tennessee_labels.csv')
df2.head()

In [10]:
incidents=pd.read_parquet('nfd_incidents_xd_seg.parquet')
incident_geo = gd.GeoDataFrame(
    incidents,
    geometry=gd.points_from_xy(incidents.longitude, incidents.latitude),
    crs=df.crs
)

In [11]:
jdi = gd.sjoin(df, incident_geo, how='inner')
len(jdi)

29768

In [ ]:
jdi = jdi.astype({'TRACTCE': 'int64', 'COUNTYFP': 'int64'})
jdi.iloc[[0]]

In [ ]:
df2_clean = df2.astype({'tract': 'int64', 'county': 'int64'}).drop(['state'], axis=1)
df2_clean.iloc[[0]]

In [17]:
df_comp = jdi.merge(df2_clean, how='left', left_on=['TRACTCE','COUNTYFP'], right_on=['tract','county'])
df_comp.iloc[[0]]

,STATEFP,COUNTYFP,TRACTCE,GEOID,NAME_x,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,...,occupied housing units,owner occupied housing units,housing units btw 500k and 750k,housing units btw 750k and 1m,housing units above 1m,vacant housing units,median housing value,median gross rent,county,tract
0,47,21,70103,47021070103,701.03,Census Tract 701.03,G5020,S,43484202,2904,...,1338,1097,22,0,0,153,152300,841,21,70103


In [ ]:
df_comp.to_file('/content/incident_census_join_econ', driver="GeoJSON")

In [ ]:
s3.upload_file(Bucket='wwu-incidents',Key='incident_census_join_econ',Filename='incident_census_join_econ')